In [21]:
print("Stack fitting Jupyter module.")

Stack fitting Jupyter module.


Test to include latex math: $\mathrm{F_\lambda\ [erg\, s^{-1}\, cm^{-2}\, \AA]} 
= \frac{L \times L_\odot}{4 \pi d_l^2}$

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

import emcee
import corner

import time

# Check that it imported the correct matplotlibrc
import matplotlib
print("Using matplotlibrc:", matplotlib.matplotlib_fname())

# To get figures to show up as a pop up
%matplotlib widget

# To get figures to show up within the notebook
# %matplotlib inline

import os
import sys

home = os.getenv('HOME')
figs_dir = home + "/Documents/pears_figs_data/"
stacking_analysis_dir = home + "/Documents/GitHub/stacking-analysis-pears/"
stacking_figures_dir = home + "/Documents/stacking_figures/"
massive_galaxies_dir = home + "/Documents/GitHub/massive-galaxies/"
pears_spectra_dir = home + "/Documents/pears_figs_data/data_spectra_only/"


Using matplotlibrc: /Users/baj/miniconda3/envs/astroconda37/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc


In [23]:
# Define redshift range 
z_low = 0.16
z_high = 0.96

In [24]:
# Load in saved stack
stack = np.genfromtxt(stacking_analysis_dir + 'massive_stack_pears_' + str(z_low) + 'z' + str(z_high) + '.txt', \
                      dtype=None, names=['lam', 'flam', 'flam_err'], encoding='ascii')

In [25]:
# Load models
models_llam = np.load(figs_dir + 'model_comp_spec_llam_withlines_chabrier.npy', mmap_mode='r')
models_grid = np.load(figs_dir + 'model_lam_grid_withlines_chabrier.npy', mmap_mode='r')

total_models = len(models_llam)
print("Total models:", total_models)

Total models: 37761


In [26]:
# Loop over all models and modify them
# ------------ NO COVARIANCE MATRIX FOR THE MOMENT ------------- # 
# First do the convolution with the LSF

# LSF defined as a Gaussian for now
lsf_sigma = 20.0  # this is in Angstroms?

models_lsfconv = np.zeros(models_llam.shape)

t1 = time.time()

for i in range(total_models):
    models_lsfconv[i] = scipy.ndimage.gaussian_filter1d(input=models_llam[i], sigma=lsf_sigma)

t2 = time.time()
print("Time taken for for loop:", "{:.2f}".format(t2 - t1), "seconds.")
    
# Without for loop over all models
models_lsfconv_nofor = scipy.ndimage.gaussian_filter1d(input=models_llam, sigma=lsf_sigma, axis=1)

t3 = time.time()
print("Time taken for vectorized computation:", "{:.2f}".format(t3 - t2), "seconds.")

print("Arrays equal:", np.array_equal(models_lsfconv_nofor, models_lsfconv))

t4 = time.time()
print("Time taken for array comparison:", "{:.2f}".format(t4 - t3), "seconds.")

print("LSF convolution done.")


Time taken for for loop: 107.98 seconds.
Time taken for vectorized computation: 153.82 seconds.
Arrays equal: True
Time taken for array comparison: 36.56 seconds.
LSF convolution done.


In [27]:
print(models_lsfconv.shape)

(37761, 13228)


In [ ]:
# Now do the resampling
# Define array to save modified models
resampling_grid = stack['lam']
models_mod = np.zeros((total_models, len(resampling_grid)))

### Zeroth element
lam_step = resampling_grid[1] - resampling_grid[0]
idx = np.where((models_grid >= resampling_grid[0] - lam_step) & \
               (models_grid < resampling_grid[0] + lam_step))[0]
models_mod[:, 0] = np.mean(models_lsfconv[:, idx], axis=1)

### all elements in between
for j in range(1, len(resampling_grid) - 1):
    idx = np.where((models_grid >= resampling_grid[j-1]) & \
                   (models_grid < resampling_grid[j+1]))[0]
    models_mod[:, j] = np.mean(models_lsfconv[:, idx], axis=1)

### Last element
lam_step = resampling_grid[-1] - resampling_grid[-2]
idx = np.where((models_grid >= resampling_grid[-1] - lam_step) & \
               (models_grid < resampling_grid[-1] + lam_step))[0]
models_mod[:, -1] = np.mean(models_lsfconv[:, idx], axis=1)

print("Resampling done.")

In [ ]:
print(models_mod.shape)

In [ ]:
# For all models fit a polynomial and divide continuum
models_divcont = np.zeros(models_mod.shape)

for k in range(total_models):
    np_fit = np.polyfit(resampling_grid, models_mod[k], deg=3)
    np_polynomial = np.poly1d(np_fit)
    
    models_divcont[k] = models_mod[k] / np_polynomial(resampling_grid)

In [ ]:
# Compute chi2 for each model
chi2 = (models_divcont - stack['flam'])**2 / stack['flam_err']**2
chi2 = np.nansum(chi2, axis=1)

In [ ]:
best_fit_idx = np.argmin(chi2)

In [ ]:
# Check by plotting best fit model
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(resampling_grid, stack['flam'], '.-', color='mediumblue', linewidth=1.5, \
        markeredgecolor='mediumblue', markersize=1.0, zorder=5)
ax.fill_between(resampling_grid, stack['flam'] - stack['flam_err'], stack['flam'] + stack['flam_err'], \
                color='gray', alpha=0.5, zorder=5)

ax.axhline(y=1.0, ls='--', color='k')

# Now plot best fitting model
ax.plot(resampling_grid, models_divcont[best_fit_idx], color='tab:red', linewidth=1.5, \
        markeredgecolor='mediumblue', markersize=1.0, zorder=5)

plt.show()